In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
my_url = 'http://www.soccer-db.info/index.php?option=com_php&views=php&Itemid=224&team1=0&team2=0&tour=all&limitstart=0'
#opening the connection and grabbing the content
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

In [3]:
page_soup = soup(page_html, "html.parser")

In [4]:
table = page_soup.findAll('table')[2]
table

<table border="0" cellpadding="3" cellspacing="2" width="100%">
<tr style="background-color:#DADEE0; text-align:center;">
<td>Date</td>
<td>Venue</td>
<td>Match</td>
<td>Score</td>
<td>Tournament</td>
<td>Season</td>
<td>Link</td>
</tr>
<tr style="background-color:#ffffff;">
<td style="text-align:center;">2018-06-19</td>
<td style="text-align:center;">Saransk, Russia</td>
<td style="text-align:center;">
<a href="/index.php?option=com_joomsport&amp;task=team&amp;tid=49&amp;sid=818" target="_blank">Colombia</a> - <a href="/index.php?option=com_joomsport&amp;task=team&amp;tid=13&amp;sid=818" target="_blank">Japan</a>
</td>
<td style="text-align:center;">1:2</td>
<td style="text-align:center;">FIFA World Cup</td>
<td style="text-align:center;"><a href="/index.php?option=com_joomsport&amp;view=ltable&amp;sid=818&amp;gr_id=0" target="_blank">2018</a></td>
<td style="text-align:center;"><a href="/index.php?option=com_joomsport&amp;task=view_match&amp;id=46064" target="_blank">Details»</a></td

In [5]:
import pandas as pd
import numpy as np
new_table = pd.DataFrame(columns=range(0,7), index=np.arange(51))

In [6]:
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        if column_marker == 6 and row_marker != 0:
            new_table.iat[row_marker,column_marker] = 'http://www.soccer-db.info' + column.a.get('href')
        else:
            new_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if row_marker == 0:
        new_table.columns = new_table.loc[0,:]
    row_marker += 1
new_table.drop([0], inplace =True)
new_table.head()

,Date,Venue,Match,Score,Tournament,Season,Link
1,2018-06-19,"Saransk, Russia",\nColombia - Japan\n,1:2,FIFA World Cup,2018,http://www.soccer-db.info/index.php?option=com...
2,2018-06-18,"Nizhny Novgorod, Russia",\nSweden - Korea Republic\n,1:0,FIFA World Cup,2018,http://www.soccer-db.info/index.php?option=com...
3,2018-06-18,"Sochi, Russia",\nBelgium - Panama\n,3:0,FIFA World Cup,2018,http://www.soccer-db.info/index.php?option=com...
4,2018-06-18,"Volgograd, Russia",\nTunisia - England\n,1:2,FIFA World Cup,2018,http://www.soccer-db.info/index.php?option=com...
5,2018-06-17,"Rostov-on-Don, Russia",\nBrazil - Switzerland\n,1:1,FIFA World Cup,2018,http://www.soccer-db.info/index.php?option=com...


In [7]:
for i, link in enumerate(new_table.loc[1, ['Link']]):
    print(i)
    print(link)
    season = new_table.loc[i+1, ['Season']][0]
    home = new_table.loc[i+1, ['Match']].str.split(' - ')[0][0][1:]
    away = new_table.loc[i+1, ['Match']].str.split(' - ')[0][1][:-1]
    print(season, home, away)
    uClient = uReq(link)
    page_html = uClient.read()
    uClient.close()
    page_soup_detail = soup(page_html, "html.parser")

    teamplayers = page_soup_detail.findAll('div', {'id':'etab_squad_div'})[0]

0
http://www.soccer-db.info/index.php?option=com_joomsport&task=view_match&id=46064
2018 Colombia Japan


In [8]:
teamplayer_table = pd.DataFrame(columns=range(0,3), index=np.arange(12))
teamplayer_table[0] = season
teamplayer_table.columns = ['Year', home, away]
teamplayer_table

,Year,Colombia,Japan
0,2018,NaN,NaN
1,2018,NaN,NaN
2,2018,NaN,NaN
3,2018,NaN,NaN
4,2018,NaN,NaN
5,2018,NaN,NaN
6,2018,NaN,NaN
7,2018,NaN,NaN
8,2018,NaN,NaN
9,2018,NaN,NaN


In [9]:
row_marker = 0
for row in teamplayers.find_all('tr'):
    column_marker = 1
    columns = row.find_all('td')
    for column in columns:
        teamplayer_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1
teamplayer_table.drop([11], inplace=True)
teamplayer_table

,Year,Colombia,Japan
0,2018,David Ospina,Eiji Kawashima
1,2018,Davinson Sánchez,Gen Shōji
2,2018,Óscar Murillo,Hiroki Sakai
3,2018,Santiago Arias,Maya Yoshida
4,2018,Carlos Alberto Sánchez,Yūto Nagatomo
5,2018,Jefferson Lerma,Gaku Shibasaki
6,2018,Johan Mojica,Makoto Hasebe
7,2018,Juan Fernando Quintero,Shinji Kagawa
8,2018,Juan Guillermo Cuadrado,Takashi Inui
9,2018,José Izquierdo,Genki Haraguchi


In [10]:
teamplayer_table = teamplayer_table.melt('Year', var_name='Country', value_name='Player_Name')
teamplayer_table

,Year,Country,Player_Name
0,2018,Colombia,David Ospina
1,2018,Colombia,Davinson Sánchez
2,2018,Colombia,Óscar Murillo
3,2018,Colombia,Santiago Arias
4,2018,Colombia,Carlos Alberto Sánchez
5,2018,Colombia,Jefferson Lerma
6,2018,Colombia,Johan Mojica
7,2018,Colombia,Juan Fernando Quintero
8,2018,Colombia,Juan Guillermo Cuadrado
9,2018,Colombia,José Izquierdo
